In [13]:
# i want to train a mlp to classify the mnist dataset
# i will use the torch library

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np
import matplotlib
import matplotlib.pyplot as plt


# define the network
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


# define the training function
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
    



In [14]:
# load the data

# define the batch size

batch_size = 64

# define the training and test datasets

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

# define the training and test dataloaders


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



100.0%
100.0%
100.0%
100.0%


In [18]:
# load the model

model = MLP()

# define the device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.to(device)

# define the optimizer

optimizer = optim.Adam(model.parameters(), lr=0.001)

# train the model

for epoch in range(1, 11):
    train(model, device, train_loader, optimizer, epoch)

# test the model

model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))



Train Epoch: 1 [0/60000 (0%)]	Loss: 2.302175
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.350215
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.235051
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.377218
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.167971
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.099581
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.183188
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.158153
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.074465
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.057925
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.113625
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.138467
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.020691
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.068514
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.019043
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.136797
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.005300
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.026533
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.025923
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.096547
Train Epoch:

In [ ]:
# now i want to compare between intialization methods ie xavier and zero initialization

# define the network

class MLP_Xavier(nn.Module):
    def __init__(self):
        super(MLP_Xavier, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    

# define the network

class MLP_Zero(nn.Module):
    def __init__(self):
        super(MLP_Zero, self).__init__()
        self.fc1 = nn.Linear(28*28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)
        nn.init.zeros_(self.fc1.weight)
        nn.init.zeros_(self.fc2.weight)
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)


# load the models


model_xavier = MLP_Xavier()
model_zero = MLP_Zero()

model_xavier.to(device)
model_zero.to(device)

# define the optimizers

optimizer_xavier = optim.Adam(model_xavier.parameters(), lr=0.001)
optimizer_zero = optim.Adam(model_zero.parameters(), lr=0.001)

# train the models

import stat
import time

start = time.time()
for epoch in range(1, 11):
    train(model_xavier, device, train_loader, optimizer_xavier, epoch)
end = time.time()
print('Time taken to train the model with xavier initialization: ', end-start)
time_xavier = end-start
start = time.time()
for epoch in range(1, 11):
    train(model_zero, device, train_loader, optimizer_zero, epoch)
end = time.time()
print('Time taken to train the model with zero initialization: ', end-start)
time_zero = end-start
# test the models

model_xavier.eval()
model_zero.eval()
test_loss_xavier = 0
test_loss_zero = 0
correct_xavier = 0
correct_zero = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output_xavier = model_xavier(data)
        output_zero = model_zero(data)
        test_loss_xavier += F.nll_loss(output_xavier, target, reduction='sum').item()
        test_loss_zero += F.nll_loss(output_zero, target, reduction='sum').item()
        pred_xavier = output_xavier.argmax(dim=1, keepdim=True)
        pred_zero = output_zero.argmax(dim=1, keepdim=True)
        correct_xavier += pred_xavier.eq(target.view_as(pred_xavier)).sum().item()
        correct_zero += pred_zero.eq(target.view_as(pred_zero)).sum().item()

test_loss_xavier /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss_xavier, correct_xavier, len(test_loader.dataset),
    100. * correct_xavier / len(test_loader.dataset)))

test_loss_zero /= len(test_loader.dataset)

print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss_zero, correct_zero, len(test_loader.dataset),
    100. * correct_zero / len(test_loader.dataset)))


print('Time taken to train the model with xavier initialization: ', time_xavier)
print('Time taken to train the model with zero initialization: ', time_zero)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.273345
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.261519
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.364717
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.095610
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.115224
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.159456
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.133708
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.108802
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.057845
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.060870
Train Epoch: 2 [0/60000 (0%)]	Loss: 0.022645
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.080956
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.055946
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.028040
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.039197
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.124865
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.035604
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.079903
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.093539
Train Epoch: 2 [57600/60000 (96%)]	Loss: 0.016214
Train Epoch: